In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import random
import shutil
import argparse
import importlib
import data_utils_legacy as data_utils
import numpy as np
import pointfly as pf
import tensorflow as tf
from datetime import datetime

import h5py

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--filelist', '-t', default="data/KITTI/ImageSets/train.txt", help='Path to training set ground truth (.txt)')
parser.add_argument('--filelist_val', '-v', default="data/KITTI/ImageSets/train.txt", help='Path to validation set ground truth (.txt)')
parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
parser.add_argument('--save_folder', '-s', default="/home/kartik/saved/", help='Path to folder for saving check points and summary')
parser.add_argument('--model', '-m', default="pointcnn_seg", help='Model to use')
parser.add_argument('--setting', '-x', default="kitti3d_x8_2048_fps_mod1", help='Setting to use')
parser.add_argument('--epochs', default="1",help='Number of training epochs (default defined in setting)', type=int)
parser.add_argument('--batch_size', default="1", help='Batch size (default defined in setting)', type=int)
# default="64",
parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')
args = parser.parse_args([])

print(args)

Namespace(batch_size=1, epochs=1, filelist='data/KITTI/ImageSets/train.txt', filelist_val='data/KITTI/ImageSets/train.txt', load_ckpt=None, log='log.txt', model='pointcnn_seg', no_code_backup=False, no_timestamp_folder=False, save_folder='/home/kartik/saved/', setting='kitti3d_x8_2048_fps_mod1')


In [3]:
cwd = os.getcwd()
model = importlib.import_module(args.model)
setting_path = os.path.join(cwd, args.model)
sys.path.append(setting_path)
setting = importlib.import_module(args.setting)

num_epochs = args.epochs or setting.num_epochs
batch_size = args.batch_size or setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
label_weights_list = setting.label_weights
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val

In [4]:
points = []
labels = []
point_nums = []
labels_seg = []
indices_split_to_full = []

print('{}-Preparing datasets...'.format(datetime.now()))
# is_list_of_h5_list = data_utils.is_h5_list(args.filelist)
args.filelist = os.path.join(cwd, "data", "Argo_h5_GR_scaled_depth", "train")+"/train.txt"
args.filelist_val = os.path.join(cwd, "data", "Argo_h5_GR_scaled_depth", "train")+"/val.txt"

is_list_of_h5_list = not data_utils.is_h5_list(args.filelist)
if is_list_of_h5_list:
    seg_list = data_utils.load_seg_list(args.filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train = args.filelist
data_train, _, data_num_train, label_train, _ = data_utils.load_seg_Argo(filelist_train)
data_val, _, data_num_val, label_val, _ = data_utils.load_seg_Argo(args.filelist_val)


# data_train = np.dstack([data_train, label_train[:, :,np.newaxis]])
# data_val = np.dstack([data_val, label_val[:, :,np.newaxis]])

# depth = np.linalg.norm(data_train, 2, axis=2)
# data_train = np.concatenate([data_train, depth[:,:,np.newaxis]], axis = -1)

# depth = np.linalg.norm(data_val, 2, axis=2)
# data_val = np.concatenate([data_val, depth[:,:,np.newaxis]], axis = -1)

# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])

0it [00:00, ?it/s]

2020-01-08 13:13:13.307866-Preparing datasets...


20it [00:05,  3.84it/s]


amax [[[217.02274   84.14794  194.86005 ]
  [216.58429   59.07181  190.29214 ]
  [218.70334   82.557396 199.96799 ]
  ...
  [143.4347    13.510128  61.26613 ]
  [131.37718   10.249168 111.17418 ]
  [ 53.269745   7.663868 111.337364]]]
amin [[[-213.55579      -7.4701376  -212.08127   ]
  [-212.70149      -3.2587485  -211.15894   ]
  [-215.07878      -5.7737727  -209.11053   ]
  ...
  [ -74.75722      -0.47247314  -64.9187    ]
  [ -98.06576      -0.66090035 -173.12178   ]
  [ -82.89937      -0.5216615   -47.190056  ]]]


1it [00:00,  4.89it/s]


amax [[[215.90323   20.952639  94.15194 ]
  [155.82857   18.881943  94.48389 ]
  [162.71529   21.812824  94.183495]
  ...
  [  0.         0.         0.      ]
  [  0.         0.         0.      ]
  [  0.         0.         0.      ]]]
amin [[[-140.0768       -0.49999964 -119.15227   ]
  [-192.31703      -1.0448203  -117.7571    ]
  [-162.12273      -0.57638097 -123.3467    ]
  ...
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]]]


In [5]:
num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))
batch_num_val = math.ceil(num_val / batch_size)
print('{}-{:d} testing batches per test.'.format(datetime.now(), batch_num_val))

2020-01-08 13:13:40.129622-43942/1639 training/validation samples.
2020-01-08 13:13:40.129732-43942 training batches.
2020-01-08 13:13:40.129856-1639 testing batches per test.


In [6]:
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
features_augmented = None

In [7]:
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled
points_augmented = pf.augment(points_sampled, xforms, jitter_range)


In [8]:
labels_sampled = tf.gather_nd(labels_seg, indices=indices, name='labels_sampled')
labels_weights_sampled = tf.gather_nd(labels_weights, indices=indices, name='labels_weight_sampled')

net = model.Net(points_augmented, features_augmented, is_training, setting)
logits = net.logits
probs = tf.nn.softmax(logits, name='probs')
predictions = tf.argmax(probs, axis=-1, name='predictions')

loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_sampled, logits=logits,
                                                 weights=labels_weights_sampled)

with tf.name_scope('metrics'):
    loss_mean_op, loss_mean_update_op = tf.metrics.mean(loss_op)
    t_1_acc_op, t_1_acc_update_op = tf.metrics.accuracy(labels_sampled, predictions, weights=labels_weights_sampled)
    t_1_per_class_acc_op, t_1_per_class_acc_update_op = \
        tf.metrics.mean_per_class_accuracy(labels_sampled, predictions, setting.num_class,
                                           weights=labels_weights_sampled)
reset_metrics_op = tf.variables_initializer([var for var in tf.local_variables()
                                             if var.name.split('/')[0] == 'metrics'])

sampling path  /home/kartik/DL_model/PointCNN/sampling
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
_ = tf.summary.scalar('loss/train', tensor=loss_mean_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])
_ = tf.summary.scalar('t_1_per_class_acc/train', tensor=t_1_per_class_acc_op, collections=['train'])

_ = tf.summary.scalar('loss/val', tensor=loss_mean_op, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_op, collections=['val'])
_ = tf.summary.scalar('t_1_per_class_acc/val', tensor=t_1_per_class_acc_op, collections=['val'])

lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)
_ = tf.summary.scalar('learning_rate', tensor=lr_clip_op, collections=['train'])
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=setting.momentum, use_nesterov=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op + reg_loss, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

saver = tf.train.Saver(max_to_keep=None)


In [22]:
# backup all code
# if not args.no_code_backup:
#     code_folder = os.path.abspath(os.path.dirname(__file__))
#     shutil.copytree(code_folder, os.path.join(root_folder, os.path.basename(code_folder)))

root_folder = os.path.join(cwd, "outfolder_argo_gr5")
if not os.path.exists(root_folder):
    os.makedirs(root_folder)
# shutil.copytree(code_folder, os.path.join(root_folder, os.path.basename(code_folder)))

folder_ckpt = os.path.join(root_folder, 'ckpts')
if not os.path.exists(folder_ckpt):
    os.makedirs(folder_ckpt)

folder_summary = os.path.join(root_folder, '../outfolder_argo_gr4/summary/5th-trial')
if not os.path.exists(folder_summary):
    os.makedirs(folder_summary)

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


2020-01-09 00:21:13.316251-Parameter number: 11455044.


In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

with tf.Session( config=tf.ConfigProto(gpu_options=gpu_options) ) as sess:
    summaries_op = tf.summary.merge_all('train')
    summaries_val_op = tf.summary.merge_all('val')
    summary_writer = tf.summary.FileWriter(folder_summary, sess.graph)

    sess.run(init_op)

    # Load the model
    if args.load_ckpt is not None:
        saver.restore(sess, args.load_ckpt)
        print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args.load_ckpt))
    else:
        latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
        if latest_ckpt:
            print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
            saver.restore(sess, latest_ckpt)
            print('{}-Checkpoint loaded from {} (Iter {})'.format(
                datetime.now(), latest_ckpt, sess.run(global_step)))

    for batch_idx_train in range(batch_num):


        ######################################################################
        # Training
        start_idx = (batch_size * batch_idx_train) % num_train
        end_idx = min(start_idx + batch_size, num_train)
        batch_size_train = end_idx - start_idx

        points_batch = data_train[start_idx:end_idx, ...] # data number
        points_num_batch = data_num_train[start_idx:end_idx, ...] # Number of data points 
        labels_batch = label_train[start_idx:end_idx, ...] # labels of segmentation for each point
        weights_batch = np.array(label_weights_list)[labels_batch] 


        if start_idx + batch_size_train == num_train:
#             if is_list_of_h5_list:
#                 filelist_train_prev = seg_list[(seg_list_idx - 1) % len(seg_list)]
#                 filelist_train = seg_list[seg_list_idx % len(seg_list)]
#                 if filelist_train != filelist_train_prev:
#                     # Load the train data and labels 
#                     data_train, _, data_num_train, label_train, _ = data_utils.load_seg(filelist_train)
#                     num_train = data_train.shape[0]
#                 seg_list_idx = seg_list_idx + 1
            data_train, data_num_train, label_train = \
                data_utils.grouped_shuffle([data_train, data_num_train, label_train])

        offset = int(random.gauss(0, sample_num * setting.sample_num_variance))
        offset = max(offset, -sample_num * setting.sample_num_clip)
        offset = min(offset, sample_num * setting.sample_num_clip)
        sample_num_train = sample_num + offset
        xforms_np, rotations_np = pf.get_xforms(batch_size_train,
                                                rotation_range=rotation_range,
                                                scaling_range=scaling_range,
                                                order=setting.rotation_order)
        sess.run(reset_metrics_op)
        sess.run([train_op, loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                 feed_dict={
                     pts_fts: points_batch,
                     indices: pf.get_indices(batch_size_train, sample_num_train, points_num_batch),
                     xforms: xforms_np,
                     rotations: rotations_np,
                     jitter_range: np.array([jitter]),
                     labels_seg: labels_batch,
                     labels_weights: weights_batch,
                     is_training: True,
                 })
        if batch_idx_train % 10 == 0:
            loss, t_1_acc, t_1_per_class_acc, summaries, step = sess.run([loss_mean_op,
                                                                    t_1_acc_op,
                                                                    t_1_per_class_acc_op,
                                                                    summaries_op,
                                                                    global_step])
            summary_writer.add_summary(summaries, step)
            print('{}-[Train]-Iter: {:06d}  Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), step, loss, t_1_acc, t_1_per_class_acc))
            sys.stdout.flush()
        ######################################################################

        if batch_idx_train % 1500 == 0:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))        
        
        
        ######################################################################
        # Validation
        if (batch_idx_train % step_val == 0 and (batch_idx_train != 0 or args.load_ckpt is not None)) \
                or batch_idx_train == batch_num - 1:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))

            sess.run(reset_metrics_op)
            for batch_val_idx in range(batch_num_val):
                start_idx = batch_size * batch_val_idx
                end_idx = min(start_idx + batch_size, num_val)
                batch_size_val = end_idx - start_idx
                points_batch = data_val[start_idx:end_idx, ...]
                points_num_batch = data_num_val[start_idx:end_idx, ...]
                labels_batch = label_val[start_idx:end_idx, ...]
                
                weights_batch = np.array(label_weights_list)[labels_batch]

                xforms_np, rotations_np = pf.get_xforms(batch_size_val,
                                                        rotation_range=rotation_range_val,
                                                        scaling_range=scaling_range_val,
                                                        order=setting.rotation_order)
                sess.run([loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                         feed_dict={
                             pts_fts: points_batch,
                             indices: pf.get_indices(batch_size_val, sample_num, points_num_batch),
                             xforms: xforms_np,
                             rotations: rotations_np,
                             jitter_range: np.array([jitter_val]),
                             labels_seg: labels_batch,
                             labels_weights: weights_batch,
                             is_training: False,
                         })
            loss_val, t_1_acc_val, t_1_per_class_acc_val, summaries_val, step = sess.run(
                [loss_mean_op, t_1_acc_op, t_1_per_class_acc_op, summaries_val_op, global_step])
            summary_writer.add_summary(summaries_val, step)
            print('{}-[ Val ]-Average:      Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), loss_val, t_1_acc_val, t_1_per_class_acc_val))
            sys.stdout.flush()
        ######################################################################
    print('{}-Done!'.format(datetime.now()))

2020-01-14 12:17:04.295273-Found checkpoint /home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter-250393
INFO:tensorflow:Restoring parameters from /home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter-250393
2020-01-14 12:17:04.673441-Checkpoint loaded from /home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter-250393 (Iter 250393)
2020-01-14 12:17:12.687251-[Train]-Iter: 250394  Loss: 0.0057  T-1 Acc: 0.9904  T-1 mAcc: 0.4951
INFO:tensorflow:/home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter-250394 is not in all_model_checkpoint_paths. Manually adding it.
2020-01-14 12:17:13.791300-Checkpoint saved to /home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter!
2020-01-14 12:17:14.555392-[Train]-Iter: 250404  Loss: 0.0146  T-1 Acc: 0.9881  T-1 mAcc: 0.4927
2020-01-14 12:17:15.319243-[Train]-Iter: 250414  Loss: 0.0255  T-1 Acc: 0.9683  T-1 mAcc: 0.4768
2020-01-14 12:17:16.080640-[Train]-Iter: 250424  Loss: 0.0062  T-1 Acc: 0.9888  T-1 mAcc: 0.4946
2020-01

2020-01-14 12:18:14.234316-[Train]-Iter: 251184  Loss: 0.0394  T-1 Acc: 0.9558  T-1 mAcc: 0.4740
2020-01-14 12:18:15.003907-[Train]-Iter: 251194  Loss: 0.0001  T-1 Acc: 1.0000  T-1 mAcc: 0.2500
2020-01-14 12:18:15.770986-[Train]-Iter: 251204  Loss: 0.0331  T-1 Acc: 0.9535  T-1 mAcc: 0.4725
2020-01-14 12:18:16.539148-[Train]-Iter: 251214  Loss: 0.0328  T-1 Acc: 0.9539  T-1 mAcc: 0.4753
2020-01-14 12:18:17.312572-[Train]-Iter: 251224  Loss: 0.0932  T-1 Acc: 0.5784  T-1 mAcc: 0.2767
2020-01-14 12:18:18.085839-[Train]-Iter: 251234  Loss: 0.4750  T-1 Acc: 0.2001  T-1 mAcc: 0.2359
2020-01-14 12:18:18.851533-[Train]-Iter: 251244  Loss: 0.0201  T-1 Acc: 0.9684  T-1 mAcc: 0.4817
2020-01-14 12:18:19.619000-[Train]-Iter: 251254  Loss: 0.0179  T-1 Acc: 0.9637  T-1 mAcc: 0.4814
2020-01-14 12:18:20.385532-[Train]-Iter: 251264  Loss: 0.0539  T-1 Acc: 0.8657  T-1 mAcc: 0.2164
2020-01-14 12:18:21.157078-[Train]-Iter: 251274  Loss: 0.0520  T-1 Acc: 0.9536  T-1 mAcc: 0.4632
2020-01-14 12:18:21.925681-[Tr

2020-01-14 12:19:18.233734-[Train]-Iter: 252004  Loss: 0.0195  T-1 Acc: 0.9729  T-1 mAcc: 0.4852
2020-01-14 12:19:19.003553-[Train]-Iter: 252014  Loss: 0.0959  T-1 Acc: 0.8503  T-1 mAcc: 0.4370
2020-01-14 12:19:19.773160-[Train]-Iter: 252024  Loss: 0.0275  T-1 Acc: 0.9619  T-1 mAcc: 0.4777
2020-01-14 12:19:20.547736-[Train]-Iter: 252034  Loss: 0.0291  T-1 Acc: 0.9513  T-1 mAcc: 0.4755
2020-01-14 12:19:21.322266-[Train]-Iter: 252044  Loss: 0.0116  T-1 Acc: 0.9707  T-1 mAcc: 0.4860
2020-01-14 12:19:22.092643-[Train]-Iter: 252054  Loss: 0.0615  T-1 Acc: 0.9390  T-1 mAcc: 0.4468
2020-01-14 12:19:22.859978-[Train]-Iter: 252064  Loss: 0.0569  T-1 Acc: 0.9379  T-1 mAcc: 0.4709
2020-01-14 12:19:23.630758-[Train]-Iter: 252074  Loss: 0.0001  T-1 Acc: 1.0000  T-1 mAcc: 0.2500
2020-01-14 12:19:24.402384-[Train]-Iter: 252084  Loss: 0.0216  T-1 Acc: 0.9609  T-1 mAcc: 0.4778
2020-01-14 12:19:25.171959-[Train]-Iter: 252094  Loss: 0.0231  T-1 Acc: 0.9767  T-1 mAcc: 0.4870
2020-01-14 12:19:25.941773-[Tr

2020-01-14 12:20:24.051446-[Train]-Iter: 252854  Loss: 0.0152  T-1 Acc: 0.9771  T-1 mAcc: 0.4866
2020-01-14 12:20:24.823103-[Train]-Iter: 252864  Loss: 0.0436  T-1 Acc: 0.9696  T-1 mAcc: 0.4838
2020-01-14 12:20:25.597111-[Train]-Iter: 252874  Loss: 0.0065  T-1 Acc: 0.9860  T-1 mAcc: 0.4930
2020-01-14 12:20:26.375986-[Train]-Iter: 252884  Loss: 0.0821  T-1 Acc: 0.9368  T-1 mAcc: 0.4711
2020-01-14 12:20:27.154469-[Train]-Iter: 252894  Loss: 0.0372  T-1 Acc: 0.9639  T-1 mAcc: 0.4691
2020-01-14 12:20:27.927393-[Train]-Iter: 252904  Loss: 0.0301  T-1 Acc: 0.9259  T-1 mAcc: 0.4576
2020-01-14 12:20:28.698358-[Train]-Iter: 252914  Loss: 0.0712  T-1 Acc: 0.9551  T-1 mAcc: 0.4344
2020-01-14 12:20:29.477518-[Train]-Iter: 252924  Loss: 0.0377  T-1 Acc: 0.9300  T-1 mAcc: 0.4611
2020-01-14 12:20:30.255725-[Train]-Iter: 252934  Loss: 0.0321  T-1 Acc: 0.9361  T-1 mAcc: 0.4692
2020-01-14 12:20:31.028705-[Train]-Iter: 252944  Loss: 0.0121  T-1 Acc: 0.9806  T-1 mAcc: 0.4869
2020-01-14 12:20:31.802729-[Tr

2020-01-14 12:21:28.423771-[Train]-Iter: 253674  Loss: 0.0175  T-1 Acc: 0.9777  T-1 mAcc: 0.4863
2020-01-14 12:21:29.204014-[Train]-Iter: 253684  Loss: 0.2675  T-1 Acc: 0.2739  T-1 mAcc: 0.2487
2020-01-14 12:21:29.983529-[Train]-Iter: 253694  Loss: 0.1503  T-1 Acc: 0.8285  T-1 mAcc: 0.3815
2020-01-14 12:21:30.756774-[Train]-Iter: 253704  Loss: 0.0001  T-1 Acc: 1.0000  T-1 mAcc: 0.2500
2020-01-14 12:21:31.528444-[Train]-Iter: 253714  Loss: 0.0641  T-1 Acc: 0.9553  T-1 mAcc: 0.4666
2020-01-14 12:21:32.306991-[Train]-Iter: 253724  Loss: 0.0010  T-1 Acc: 0.9976  T-1 mAcc: 0.2494
2020-01-14 12:21:33.082854-[Train]-Iter: 253734  Loss: 0.0454  T-1 Acc: 0.9675  T-1 mAcc: 0.4839
2020-01-14 12:21:33.855780-[Train]-Iter: 253744  Loss: 0.0729  T-1 Acc: 0.9229  T-1 mAcc: 0.4424
2020-01-14 12:21:34.627461-[Train]-Iter: 253754  Loss: 0.0062  T-1 Acc: 0.9859  T-1 mAcc: 0.4930
2020-01-14 12:21:35.406257-[Train]-Iter: 253764  Loss: 0.0283  T-1 Acc: 0.9756  T-1 mAcc: 0.4840
2020-01-14 12:21:36.184885-[Tr

...

2020-01-14 12:29:58.366986-[Train]-Iter: 259464  Loss: 0.0090  T-1 Acc: 0.9908  T-1 mAcc: 0.4942
2020-01-14 12:29:59.141800-[Train]-Iter: 259474  Loss: 0.0524  T-1 Acc: 0.8296  T-1 mAcc: 0.3510
2020-01-14 12:29:59.915666-[Train]-Iter: 259484  Loss: 0.0002  T-1 Acc: 1.0000  T-1 mAcc: 0.2500
2020-01-14 12:30:00.687617-[Train]-Iter: 259494  Loss: 0.0256  T-1 Acc: 0.9705  T-1 mAcc: 0.4804
2020-01-14 12:30:01.468517-[Train]-Iter: 259504  Loss: 0.0232  T-1 Acc: 0.9722  T-1 mAcc: 0.4794
2020-01-14 12:30:02.244519-[Train]-Iter: 259514  Loss: 0.0172  T-1 Acc: 0.9614  T-1 mAcc: 0.4821
2020-01-14 12:30:03.017949-[Train]-Iter: 259524  Loss: 0.0188  T-1 Acc: 0.9410  T-1 mAcc: 0.4778
2020-01-14 12:30:03.790063-[Train]-Iter: 259534  Loss: 0.0828  T-1 Acc: 0.7612  T-1 mAcc: 0.1903
2020-01-14 12:30:04.568588-[Train]-Iter: 259544  Loss: 0.0414  T-1 Acc: 0.9534  T-1 mAcc: 0.4757
2020-01-14 12:30:05.345652-[Train]-Iter: 259554  Loss: 0.1133  T-1 Acc: 0.7496  T-1 mAcc: 0.3671
2020-01-14 12:30:06.117964-[Tr

2020-01-14 12:31:04.270448-[Train]-Iter: 260314  Loss: 0.0198  T-1 Acc: 0.9745  T-1 mAcc: 0.4828
2020-01-14 12:31:05.047532-[Train]-Iter: 260324  Loss: 0.0224  T-1 Acc: 0.9686  T-1 mAcc: 0.4823
2020-01-14 12:31:05.818706-[Train]-Iter: 260334  Loss: 0.0086  T-1 Acc: 0.9705  T-1 mAcc: 0.4799
2020-01-14 12:31:06.592201-[Train]-Iter: 260344  Loss: 0.0219  T-1 Acc: 0.9693  T-1 mAcc: 0.4818
2020-01-14 12:31:07.366261-[Train]-Iter: 260354  Loss: 0.1389  T-1 Acc: 0.7593  T-1 mAcc: 0.3924
2020-01-14 12:31:08.140355-[Train]-Iter: 260364  Loss: 0.0387  T-1 Acc: 0.9751  T-1 mAcc: 0.4855
2020-01-14 12:31:08.911045-[Train]-Iter: 260374  Loss: 0.0093  T-1 Acc: 0.9899  T-1 mAcc: 0.4929
2020-01-14 12:31:09.683702-[Train]-Iter: 260384  Loss: 0.0122  T-1 Acc: 0.9795  T-1 mAcc: 0.2449
2020-01-14 12:31:10.464233-[Train]-Iter: 260394  Loss: 0.0523  T-1 Acc: 0.9585  T-1 mAcc: 0.4811
INFO:tensorflow:/home/kartik/DL_model/PointCNN/outfolder_argo_gr5/ckpts/iter-260394 is not in all_model_checkpoint_paths. Manua

2020-01-14 12:33:02.912676-[Train]-Iter: 261104  Loss: 0.0488  T-1 Acc: 0.9165  T-1 mAcc: 0.4569
2020-01-14 12:33:03.690999-[Train]-Iter: 261114  Loss: 0.0146  T-1 Acc: 0.9746  T-1 mAcc: 0.4879
2020-01-14 12:33:04.470818-[Train]-Iter: 261124  Loss: 0.0150  T-1 Acc: 0.9842  T-1 mAcc: 0.4896
2020-01-14 12:33:05.243424-[Train]-Iter: 261134  Loss: 0.0003  T-1 Acc: 0.9995  T-1 mAcc: 0.2499
2020-01-14 12:33:06.014387-[Train]-Iter: 261144  Loss: 0.0115  T-1 Acc: 0.9833  T-1 mAcc: 0.4907
2020-01-14 12:33:06.794566-[Train]-Iter: 261154  Loss: 0.0193  T-1 Acc: 0.9595  T-1 mAcc: 0.4797
2020-01-14 12:33:07.573896-[Train]-Iter: 261164  Loss: 0.0113  T-1 Acc: 0.9869  T-1 mAcc: 0.4906
2020-01-14 12:33:08.346471-[Train]-Iter: 261174  Loss: 0.0394  T-1 Acc: 0.9831  T-1 mAcc: 0.4875
2020-01-14 12:33:09.121774-[Train]-Iter: 261184  Loss: 0.0400  T-1 Acc: 0.9301  T-1 mAcc: 0.4628
2020-01-14 12:33:09.902157-[Train]-Iter: 261194  Loss: 0.0914  T-1 Acc: 0.8520  T-1 mAcc: 0.4194
2020-01-14 12:33:10.677783-[Tr

2020-01-14 12:34:08.806438-[Train]-Iter: 261954  Loss: 0.0759  T-1 Acc: 0.9343  T-1 mAcc: 0.4654
2020-01-14 12:34:09.581743-[Train]-Iter: 261964  Loss: 0.0455  T-1 Acc: 0.9350  T-1 mAcc: 0.4523
2020-01-14 12:34:10.359846-[Train]-Iter: 261974  Loss: 0.0007  T-1 Acc: 1.0000  T-1 mAcc: 0.2500
2020-01-14 12:34:11.133129-[Train]-Iter: 261984  Loss: 0.0027  T-1 Acc: 0.9995  T-1 mAcc: 0.2499
2020-01-14 12:34:11.902994-[Train]-Iter: 261994  Loss: 0.0626  T-1 Acc: 0.8149  T-1 mAcc: 0.2037
2020-01-14 12:34:12.679181-[Train]-Iter: 262004  Loss: 0.0278  T-1 Acc: 0.9784  T-1 mAcc: 0.4647
2020-01-14 12:34:13.457097-[Train]-Iter: 262014  Loss: 0.0215  T-1 Acc: 0.9743  T-1 mAcc: 0.4863
2020-01-14 12:34:14.229670-[Train]-Iter: 262024  Loss: 0.0849  T-1 Acc: 0.8145  T-1 mAcc: 0.4144
2020-01-14 12:34:15.002107-[Train]-Iter: 262034  Loss: 0.1194  T-1 Acc: 0.7802  T-1 mAcc: 0.3653
2020-01-14 12:34:15.779396-[Train]-Iter: 262044  Loss: 0.0007  T-1 Acc: 0.9995  T-1 mAcc: 0.2499
2020-01-14 12:34:16.563200-[Tr

2020-01-14 12:35:13.227490-[Train]-Iter: 262774  Loss: 0.0206  T-1 Acc: 0.9358  T-1 mAcc: 0.4698
2020-01-14 12:35:14.001009-[Train]-Iter: 262784  Loss: 0.0358  T-1 Acc: 0.9359  T-1 mAcc: 0.4631
2020-01-14 12:35:14.774599-[Train]-Iter: 262794  Loss: 0.0710  T-1 Acc: 0.9279  T-1 mAcc: 0.4598
2020-01-14 12:35:15.550966-[Train]-Iter: 262804  Loss: 0.0464  T-1 Acc: 0.9245  T-1 mAcc: 0.4650
2020-01-14 12:35:16.331295-[Train]-Iter: 262814  Loss: 0.0145  T-1 Acc: 0.9798  T-1 mAcc: 0.4902
2020-01-14 12:35:17.106313-[Train]-Iter: 262824  Loss: 0.0115  T-1 Acc: 0.9679  T-1 mAcc: 0.4893
2020-01-14 12:35:17.877934-[Train]-Iter: 262834  Loss: 0.0684  T-1 Acc: 0.8812  T-1 mAcc: 0.4314
2020-01-14 12:35:18.653155-[Train]-Iter: 262844  Loss: 0.0125  T-1 Acc: 0.9860  T-1 mAcc: 0.4926
2020-01-14 12:35:19.429029-[Train]-Iter: 262854  Loss: 0.0914  T-1 Acc: 0.8304  T-1 mAcc: 0.4068
2020-01-14 12:35:20.204631-[Train]-Iter: 262864  Loss: 0.0642  T-1 Acc: 0.9619  T-1 mAcc: 0.4324
2020-01-14 12:35:20.978944-[Tr

In [ ]:
filename_ckpt